In [1]:
from openai import AzureOpenAI
from pydantic import BaseModel
from neopipe.result import Ok, Result, Err
from neopipe.task import FunctionSyncTask, ClassSyncTask
from neopipe.pipeline import SyncPipeline
from dotenv import load_dotenv, find_dotenv
import os
import logging

In [2]:
logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)
# logging.getLogger("neopipe").setLevel(logging.ERROR)

In [3]:
load_dotenv(find_dotenv("../../dev.env"), override=True)

True

In [4]:
LLM_CLIENT = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [5]:
# ───────────────────────────────────────────────────────────────
# 1) In‑memory “database”
# ───────────────────────────────────────────────────────────────
DB = [
    {"id": 1, "text": "The quick brown fox jumps over the lazy dog"},
    {"id": 2, "text": "A journey of a thousand miles begins with a single step"},
    {"id": 3, "text": "To be or not to be, that is the question"},
]


In [6]:
class SearchResult(BaseModel):
    query: str
    hits: list[dict]

In [7]:
@FunctionSyncTask.decorator()
def search_task(res: Result[str, str]) -> Result[SearchResult, str]:
    """
    Given a query string in res.unwrap(), returns SearchResult:
    """
    if res.is_err():
        return res

    query = res.unwrap()
    hits = [record for record in DB if query.lower() in record["text"].lower()]
    return Ok(SearchResult(query=query, hits=hits))

In [8]:
search_result = search_task(Result.Ok("quick"))

2025-04-20 23:11:14 - neopipe.task - INFO - [search_task] Attempt 1 - Task ID: bd5b20c7-0dac-4a8d-820a-f23b4ea22e9d
2025-04-20 23:11:14 - neopipe.task - INFO - [search_task] Success on attempt 1


In [9]:
# search_result.unwrap().model_dump()

In [10]:
@FunctionSyncTask.decorator()
def build_context_task(res: Result[SearchResult, str]) -> Result[list[dict], str]:
    """
    Given the context dict {query,hits}, build the OpenAI chat messages:
      [ system, user, assistant(context-of-hits) ]
    """
    if res.is_err():
        return res

    ctx = res.unwrap().model_dump()
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that summarizes search results."
        },
        {
            "role": "user",
            "content": f"I’m looking for records containing: “{ctx['query']}”"
        },
        {
            "role": "assistant",
            "content": (
                f"Found {len(ctx['hits'])} record(s):\n" +
                "\n".join(f"- {h['text']}" for h in ctx["hits"])
            )
        },
    ]
    return Ok(messages)

In [11]:
build_context_task(search_result)

2025-04-20 23:11:15 - neopipe.task - INFO - [build_context_task] Attempt 1 - Task ID: 6f4013a0-7b2e-431f-bb1e-9cb4fd03a48b
2025-04-20 23:11:15 - neopipe.task - INFO - [build_context_task] Success on attempt 1


Ok([{'role': 'system', 'content': 'You are a helpful assistant that summarizes search results.'}, {'role': 'user', 'content': 'I’m looking for records containing: “quick”'}, {'role': 'assistant', 'content': 'Found 1 record(s):\n- The quick brown fox jumps over the lazy dog'}])

In [12]:
class OpenAITask(ClassSyncTask[list[dict], str]):
    """
    Sends the assembled messages list to OpenAI and returns the assistant’s reply.
    """
    def __init__(self, client: AzureOpenAI, model: str ="gpt-4o"):
        super().__init__()
        self.client = client
        self.model = model

    def execute(self, res: Result[list[dict], str]) -> Result[str, str]:
        if res.is_err():
            return res
        messages = res.unwrap()
        try:
            resp = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return Ok(resp.choices[0].message.content.strip())
        except Exception as e:
            return Err(str(e))

In [13]:
pipeline = SyncPipeline.from_tasks(
    [search_task, build_context_task, OpenAITask(client=LLM_CLIENT)],
    name="SearchAndRespond"
)

In [14]:
def run_search_and_respond(user_query: str):
    # Wrap the user query in Ok(...)
    result = pipeline.run(Ok(user_query), debug=True)

    if result.is_err():
        print("❌ Pipeline error:", result.err())
    else:
        final, trace = result.unwrap()
        print("🤖 AI Response:\n", final, "\n")
        print("—-- Trace —--")
        print(f"Input query: {user_query!r}")
        for task_name, step_res in trace:
            print(f"{task_name:20} → {step_res}")

In [15]:
q = "quick"
run_search_and_respond(q)

2025-04-20 23:11:16 - neopipe.pipeline - INFO - [SearchAndRespond] Starting with 3 task(s)
2025-04-20 23:11:16 - neopipe.pipeline - INFO - [SearchAndRespond] Task 1/3 → search_task
2025-04-20 23:11:16 - neopipe.task - INFO - [search_task] Attempt 1 - Task ID: bd5b20c7-0dac-4a8d-820a-f23b4ea22e9d
2025-04-20 23:11:16 - neopipe.task - INFO - [search_task] Success on attempt 1
2025-04-20 23:11:16 - neopipe.pipeline - INFO - [SearchAndRespond] Task 2/3 → build_context_task
2025-04-20 23:11:16 - neopipe.task - INFO - [build_context_task] Attempt 1 - Task ID: 6f4013a0-7b2e-431f-bb1e-9cb4fd03a48b
2025-04-20 23:11:16 - neopipe.task - INFO - [build_context_task] Success on attempt 1
2025-04-20 23:11:16 - neopipe.pipeline - INFO - [SearchAndRespond] Task 3/3 → OpenAITask
2025-04-20 23:11:16 - neopipe.task - INFO - [OpenAITask] Attempt 1 - Task ID: b73b683b-a58e-49b6-9a34-b53c13c1216d
2025-04-20 23:11:17 - neopipe.task - INFO - [OpenAITask] Success on attempt 1
2025-04-20 23:11:17 - neopipe.pipeli

🤖 AI Response:
 Found the following record:

- "The quick brown fox jumps over the lazy dog." This phrase is a well-known example in typography and typewriter tests because it contains all the letters of the English alphabet. 

—-- Trace —--
Input query: 'quick'
SearchAndRespond     → Ok('quick')
search_task          → Ok(SearchResult(query='quick', hits=[{'id': 1, 'text': 'The quick brown fox jumps over the lazy dog'}]))
build_context_task   → Ok([{'role': 'system', 'content': 'You are a helpful assistant that summarizes search results.'}, {'role': 'user', 'content': 'I’m looking for records containing: “quick”'}, {'role': 'assistant', 'content': 'Found 1 record(s):\n- The quick brown fox jumps over the lazy dog'}])
OpenAITask           → Ok('Found the following record:\n\n- "The quick brown fox jumps over the lazy dog." This phrase is a well-known example in typography and typewriter tests because it contains all the letters of the English alphabet.')
